# Tools

In [202]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd

pd.options.display.max_columns=None
pd.set_option('max_colwidth', None)  # 設定 value 顯示長度

# 開爬
> **爬取每日熱門爬取每日熱門文章。**

> **爬取資訊**
    >- 看板
    >- 文章標題
    >- 作者
    >- 發文時間
    >- 文章連結
    >- 推文數
    >- 文章內容
    >- 推文內容

> **資訊位置**
    >- 'div', 'r-ent'
    - 文章標題

>- 流程
    1. 輸入URL，獲取網頁原始碼
    2. 透過 BeautifulSoup 解析 HTML 格式文件
    3. 從最新頁面開始爬取
        - 設定爬取條件
        - 將需求資訊以 dict 儲存
        - return 上一頁 URL
    4. 往上一頁爬取（設定爬取條件）

In [136]:
url = "https://www.ptt.cc/bbs/NBA/index.html"

### 1. 取得 response
r = requests.get(url)  # cookies={'over18': '1'}

### 2. 透過 parser 轉換成 BeautifulSoup 比較好處理的格式（以 BeautifulSoup 解析 HTML 格式文件）
"""
    - soup是獲得的文件物件, html.parser是支援的parser.
    - 關於parser, 比較常見的parser有這些：html.parser / html5lib / lxml。
    
    - BeautifulSoup 會根據指定的 parser 把文件轉換成一個複雜的樹形結構，每個節點都是 Python 物件。
        - 所有物件大致上可歸類為4種:
            - BeautifulSoup: 表示一個文件的所有內容.
            - Tag: 基本上就是HTML裡的各種tag(div, p, etc...). 每個tag有兩個重要的屬性: 
                - name: 即tag的名字或著tag本身的name.
                - attrs: 通常指一個tag的class(常用).
            - NavigableString: 取得tag內的文字, 如: soup.p.string
            - Comment: 此物件是一個特殊的NavigableString物件, 其輸出的內容不包含註解符號.
"""
soup = BeautifulSoup(r.text, "html.parser")

### 3. Results
print('status_code: ', r.status_code)
# print('未解析資料: ', r.text)
print('解析後資料', soup)

status_code:  200
解析後資料 <!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>看板 NBA 文章列表 - 批踢踢實業坊</title>
<link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-custom.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/NBA/index.html"><span class="board-label">看板 </span>NBA</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</di

## 文章標題

In [37]:
# 尋找 class="title" 的 div 標籤
titles = soup.find('div', class_='title')
print('標題文字：', titles.a.string)

標題文字： [花邊] 公牛吉祥物秀中場投籃 不滿場邊暴龍干擾


In [38]:
# 尋找所有 class="title" 的 div 標籤
titles = soup.find_all('div', class_='title')
for title in titles:
    print(title.a.string)

[花邊] 公牛吉祥物秀中場投籃 不滿場邊暴龍干擾
[花邊] 希望KI打主場? 鬍子:我會幫他打疫苗
[情報] NBA Standings (Jan. 13, 2022)
[新聞] 國王播歌嘲諷「冷如冰」 Westbrook:很有趣
[公告] 水桶 
[新聞] 葛蘭德初獲大三元 騎士掌握制空權痛擊爵
[花邊] 籃網三巨頭合體威力驚人 哈登:就是這麼出
[公告] 板規v9.0
[情報] SEASON Schedule January 21–22
[情報] NBA Standings (Jan. 13, 2022)


## test

In [137]:
# 爬取整篇文章
articles = soup.find_all('div', 'r-ent')

for article in articles:
    PTT_URL = "https://www.ptt.cc"
    page_div = soup.find('div', 'btn-group btn-group-paging')
    prev_page = page_div.find_all('a')[1]['href']  # 上頁
    latest_page = page_div.find_all('a')[3]['href']  # 最新
    
    today = dt.datetime.today().date().strftime('%m/%d').lstrip('0')
    date = article.find('div', 'date').text.strip()  # strip(): 去除字串頭尾的空格
    
    if date == today:
        if article.find('div', 'title').a == None:
            print(None)
        else:
            print(article.find('div', 'title').a.text)
    
    r = requests.get(PTT_URL + prev_page)
    soup = BeautifulSoup(r.text, "html.parser")

None
None
[花邊] KD比賽中想偷偷跑上場換人被Nash拉住
Re: [討論] Curry和KD哪個進攻比較難守
[花邊] 金塊Bol Bol交易取消
Re: [花邊] Booker跟裁判抗議罰球時被暴龍的吉祥物干
[花邊] 全明星第二階段票數公佈
[情報] 聯盟禁止國王主場繼續頒發單場最冷球員
[花邊] LBJ終於有滿格Wi-Fi 了
[Live] 勇士 @ 公鹿
[Live] 快艇 @ 鵜鶘
[Live] 灰狼 @ 灰熊
[外絮] 龜龜：把批評視為讚美，如果受到關注表
[情報] 湖人正在評估DH,DAJ,Bazemore的交易價值
[Live] 雷霆 @ 籃網
[花邊] 球迷拿Zion球衣換Ja球衣
[討論] 有底薪上車結果被當成籌碼去換人的例子嗎
[Live] 拓荒者 @ 金塊
[BOX ] Warriors 99:118 Bucks 數據
[外絮] 尼克或許會在本季交易JR


In [155]:
single_article = soup.find_all('div', 'r-ent')[2]

# 需求資訊
title = single_article.find('div', 'title').a.text  # 標題
push_count = single_article.find('div', 'nrec').text  # 推文數 (str)
author = single_article.find('div', 'author').text  # author
date = single_article.find('div', 'date').text  # Date (str)
href = single_article.find('a')['href']  # URL = https://www.ptt.cc + href
board = soup.find('a', 'board').text.lstrip('看板 ')

# 頁面操作
page_div = soup.find('div', 'btn-group btn-group-paging')
prev_page = page_div.find_all('a')[1]['href']  # 上頁
latest_page = page_div.find_all('a')[3]['href']  # 最新
board

'NBA'

In [134]:
date = single_article.find('div', 'date').text
dt.datetime.today().date().strftime('%m/%d').lstrip('0') == date.strip()
dt.datetime.today().date().strftime('%m/%d').lstrip('0')

'1/14'

## modules
> **爬取資訊**
    >- 看板
    >- 文章標題
    >- 作者
    >- 發文時間
    >- 文章連結
    >- 推文數
    >- 文章內容
    >- 推文內容

In [171]:
def get_webPage(url):
    r = requests.get(url=url, cookies={'over18': '1'})  # cookies：有些看板會擋是否已滿18歲
    if r.status_code != 200:
        print('URL 無效，status code = ', r.status_code)
    else:
        return r.text
    
def get_articles(page):
    # parse
    soup = BeautifulSoup(page, "html.parser")
    divs = soup.find_all('div', 'r-ent')
    
    # 頁面資訊
    PTT_URL = "https://www.ptt.cc"
    board = soup.find('a', 'board').text.lstrip('看板 ')
    page_div = soup.find('div', 'btn-group btn-group-paging')
    prev_page_url = PTT_URL + page_div.find_all('a')[1]['href']  # 上頁
#     latest_page_url = PTT_URL + page_div.find_all('a')[3]['href']  # 最新
    
    # 日期
    today = dt.datetime.today().date().strftime('%m/%d').lstrip('0')
    
    # 需求欄位
    titles=[]
    push_counts=[]
    authors=[]
    hrefs=[]
    
    # 最新頁面
    for article in divs:
        date = article.find('div', 'date').text.strip()  # strip(): 去除字串頭尾的空格
        
        # 篩選當日文章
        if date == today:
            # 排除已刪除貼文
            if article.find('div', 'title').a == None:
                continue
            else:
                # 需求資訊
                title = article.find('div', 'title').a.text  # 標題
                push_count = article.find('div', 'nrec').text  # 推文數 (str)
                author = article.find('div', 'author').text  # author
                href = PTT_URL + article.find('a')['href']  # URL = https://www.ptt.cc + href
                
                titles.append(title)
                push_counts.append(push_count)
                authors.append(author)
                hrefs.append(href)
        else:
            continue
    
    # 前一頁
    prev_page = get_webPage(prev_page_url)
        
    data = {'date': today, 
            'board': board, 
            'title': titles, 
            'push_count': push_counts, 
            'author': authors, 
            'href': hrefs}
    
    return data, prev_page_url

## 主程式
- 再包一層：get_ptt_data()

In [205]:
all_data = []
page = get_webPage("https://www.ptt.cc/bbs/NBA/index.html")
dict_data, prev_page_url = get_articles(page)
all_data.append(dict_data)

# 往前 5 頁
count = 0
while count < 5:
    page = get_webPage(prev_page_url)
    dict_data, prev_page_url = get_articles(page)
    all_data.append(dict_data)
    count += 1

df = pd.DataFrame()
for data in all_data:
    df = pd.concat([df, pd.DataFrame(data)])

In [206]:
df

,date,board,title,push_count,author,href
0,1/14,NBA,[BOX ] Clippers 89:113 Pelicans 數據,15,Rambo,https://www.ptt.cc/bbs/NBA/M.1642130109.A.040.html
1,1/14,NBA,[BOX ] Timberwolves 108:116 Grizzlies 數據,55,Rambo,https://www.ptt.cc/bbs/NBA/M.1642131342.A.E86.html
2,1/14,NBA,[花邊] Bobby Portis 慶祝動作,45,hijacknokia,https://www.ptt.cc/bbs/NBA/M.1642131497.A.1AF.html
3,1/14,NBA,[BOX ] Thunder 130:109 Nets 數據,67,Rambo,https://www.ptt.cc/bbs/NBA/M.1642132256.A.CE8.html
4,1/14,NBA,[討論] Gobert跟Green的重要性被低估了嗎,37,k7202001,https://www.ptt.cc/bbs/NBA/M.1642134299.A.362.html
5,1/14,NBA,[情報] 湖人本季對戰超過五成勝率的球隊僅五勝,51,arod1414,https://www.ptt.cc/bbs/NBA/M.1642134306.A.410.html
6,1/14,NBA,[外絮] Curry:雖然CP3至今無冠 但不影響他的偉大,36,jackie0414,https://www.ptt.cc/bbs/NBA/M.1642134811.A.5A6.html
7,1/14,NBA,Re: [花邊] 孩童拿敵隊球衣免費換Morant球衣+2張票,30,derwy,https://www.ptt.cc/bbs/NBA/M.1642135461.A.F0C.html
0,1/14,NBA,[花邊] KD比賽中想偷偷跑上場換人被Nash拉住,44,tim900127,https://www.ptt.cc/bbs/NBA/M.1642091943.A.05E.html
1,1/14,NBA,Re: [討論] Curry和KD哪個進攻比較難守,40,WeiBirdFan,https://www.ptt.cc/bbs/NBA/M.1642092216.A.8C4.html
